<a href="https://colab.research.google.com/github/Chaitrahooli/Machine-Learning-Project/blob/master/Amazon_Dataset_product_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amazon Dataset For ProductRecommendation Using Surprise Module

[Amazon review datasets](http://jmcauley.ucsd.edu/data/amazon/)

surprise module is easy-to-use Recommendation

*   The name **SurPRISE** stands for simple python Recommendation System Engine.
*   Provides the various ready to use prediction algorithm such as SVD,SVD++,KNN,PMF,NMF and also,various similarity measures (cosin, pearson,MSD)are built in.

*   Make it easy to implement the new algorithms ideas.
*   Provide the tools to evaluate,analysis and compare the algorithm performance,Cross-validation procedures can be run very easily using powerfull CV iterators(inspired by the scikit learn excellent tool )as well as exhaustive search over set of parameters.








# Recommendation system

A Recommendation system predicts the likelyhood that the user predicts that a user has prefer an item.Based on the users previous intractions with the datasource that the system takes the information from,the system is capable of recommending item to the user

**Types of Recommendation system:-** Recommendation system can be broadly classified into 3 steps those are as follows



*   **Simple recommenders:-** Offer generalized recommendation to every user,based on movie, population and genre.The basic idea behind this system is that movies that are popular and critically acclaimed will have the higher probability of being liked by the average audience.
*   **Collaborative filtering engine:-** These systems try to predict the rating or preference that the user would give an item based on the past rating and preference on other users.collaborative filters do not requires.collaborative filters donot require item metadata like its content based counterparts.


*   **Content-Based recommenders:-** suggests the similar items based on the perticular items .This system uses item metadata such as genre,director,description,actors etc.for movies make these recommendation.The general idea behind these is that if a person liked the perticular item, he or she will also like an item that is similar to it

In this project we use the collaborative filtering system,in that collaborative filtering there are two types.



*   **User-Based Collaborative Filtering:-**Imagine we want to recommend an movie to our friend.We could assume that the similar people will have the similar taste suppose me and my friend have seen the same film and rated almost similar but my friend did not seen another movie that what i already watched that film then the system will recommend that movie to my friend.

*   **Item-Based Collaborative Filtering:-** Instead of focusing on friends it will focus on that what items from all the options are more similar to what we knows he enjoys.







# Downloading the surprise module in Anaconda Nevigator

For use of this module first we have to download the module at Anaconda Nevigator using below command:

**conda install -c conda-forge scikit-surprise**

**OR**

you can use  **!pip install surprise** in your jupyter notebook

After that we have to import the surprise module in our jupyter notebook.

In [1]:
!pip install surprise

     |████████████████████████████████| 3.3MB 2.8MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.0.6-cp36-cp36m-linux_x86_64.whl size=1683523 sha256=52391ff42135c3e2a8a007df2129246971d60ae0017654fd585b335900964b34
  Stored in directory: /root/.cache/pip/wheels/ec/c0/55/3a28eab06b53c220015063ebbdb81213cd3dcbb72c088251ec
Successfully built scikit-surprise


**Importing the laibraries**

In [0]:
import surprise

**upload the files in colab notebook**

In [3]:
from google.colab import files
files.upload()

Saving reviews_Cell_Phones_and_Accessories_5.json.gz to reviews_Cell_Phones_and_Accessories_5.json.gz


In [0]:
import numpy as np
import pandas as pd


review_cell_phones_and_Accessories is a file of type **.gz** format which means it is a archive file compressed by standard GNU zip compression algorithm

we have to convert this file into normal file that means json file

In [0]:
import json
import gzip

def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.dumps(eval(l))

f = open("output.strict", 'w')
for l in parse("reviews_Cell_Phones_and_Accessories_5.json.gz"):
  f.write(l + '\n')

In [0]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Cell_Phones_and_Accessories_5.json.gz')

select the required rows from the set of rows

In [0]:
r_cols=['reviewerID','asin','overall']


In [33]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4.0,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5.0,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5.0,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4.0,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5.0,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


convert dataset into user-item intraction

In [34]:
import math
def smooth_reference(x):
    return math.log(1+x,2)
full_df=df.groupby(['reviewerID','asin'])['overall'].sum().apply(smooth_reference).reset_index()
print('user/item intraction',len(full_df))
full_df.head()

user/item intraction 194439


,reviewerID,asin,overall
0,A00126503SUWI86KZBMIN,B006FEBZRC,1.000000
1,A00126503SUWI86KZBMIN,B0090ENC6G,2.584963
2,A00126503SUWI86KZBMIN,B009LXY59E,2.584963
3,A00126503SUWI86KZBMIN,B00EB3IAAE,2.584963
4,A00126503SUWI86KZBMIN,B00F856178,2.584963


rename **reviewerID** as **uid** , **asin** as **iid** and **overall** as **rating**

In [0]:
dataset=full_df.rename(columns={'reviewerID':'uid','asin':'iid','overall':'rating'},inplace=False).head()

In [36]:
dataset.head()

,uid,iid,rating
0,A00126503SUWI86KZBMIN,B006FEBZRC,1.000000
1,A00126503SUWI86KZBMIN,B0090ENC6G,2.584963
2,A00126503SUWI86KZBMIN,B009LXY59E,2.584963
3,A00126503SUWI86KZBMIN,B00EB3IAAE,2.584963
4,A00126503SUWI86KZBMIN,B00F856178,2.584963


find out the lower rating and upper rating

In [37]:
lower_rating=dataset['rating'].min()
upper_rating=dataset['rating'].max()
print('Review rate {0} to {1}'.format(lower_rating,upper_rating))

Review rate 1.0 to 2.584962500721156


review range is from 1 to 2.5

then we load our dataset

In [0]:
reader=surprise.Reader(rating_scale=(1.0,2.5))
data=surprise.Dataset.load_from_df(dataset,reader)

we will use the method SVD++ one of the best performer in the Netflix challenge,which has now become a popular method for fitting the recommendation system.

now we have trained the model on the whole datasets

In [0]:
alg=surprise.SVDpp()
output=alg.fit(data.build_full_trainset())

now we fitted the model to cheack the prediction score for the userid is 12 and item id is BOOGM5NA36

In [0]:
pred=alg.predict(uid='12',iid='BOOGM5NA36')

find out the estimated score in this case the estimated score is 2.26

In [41]:
score=pred.est
print(score)

2.2679700005769248


we need to find out the item id that user didnot rate

In [0]:
#get the list of all movie ids
iids=dataset['iid'].unique()
#get the list if iids that uid  has rated
iid_user=dataset.loc[dataset['uid']==12,'iid']
#remove the iids that uid has rated from the list of all movie ids
iids_to_pred=np.setdiff1d(iids,iid_user)

next we want to predict the score of each movie ids that user has not rate and find the best one for this we want to create the another dataset with the iids we want to predict in the sparse format  as before of:uid,iid and rating.we'll arbitrarily set all the ratings of the test set to 2.5,then the output of the first prediction

In [43]:
testset=[[12,iid,2.5]for iid in iids_to_pred]
predictions=alg.test(testset)
predictions[0]

Prediction(uid=12, iid='B006FEBZRC', r_ui=2.5, est=2.099038882375386, details={'was_impossible': False})

here each prediction is a special object we'll convert this object into an array of predicted ratings,we'll then use this to find the iid with the best predicted ratings 

In [44]:
pred_ratings=np.array([pred.est for pred in predictions])
#find the index of the maximum predicted rating
i_max=pred_ratings.argmax()
iid=iids_to_pred[i_max]
#use this to find the corresponding iid to recommend
print('Top item for the user 12 has iid {0} with prediction rating {1}'.format(iid,pred_ratings[i_max]))

Top item for the user 12 has iid B00EB3IAAE with prediction rating 2.323785774266262


when you implement your own recommedation system you will normally have the metadata which allows you to get for example name of the product from the iid code unfortunately this dataset doesnot include  product name information.

then you can get the topn items from the user,for this you have to just replace argmax() to argpartition()

In [45]:
#get top n item for the user
top_max=pred_ratings.argpartition(kth=1)
top_iid=iids_to_pred[top_max]
print('Top n item for the user 12 has iid {0} with prediction rating {1}'.format(top_iid,pred_ratings[top_max]))

Top n item for the user 12 has iid ['B006FEBZRC' 'B0090ENC6G' 'B009LXY59E' 'B00EB3IAAE' 'B00F856178'] with prediction rating [2.09903888 2.2966758  2.30872667 2.32378577 2.31181965]


it will recommend the topn items for the userid 12